In [1]:
# imports
import os
import glob
import numpy as np
import pandas as pd 
import altair as alt
# import altair_viewer

In [2]:
# load in dataframes and concatenate together
path = (
    "C:/Users/jacob.hester/Documents/Python_scripts/"\
    "Historical_beef_cattle_prices/data/processed"
    )
raw_files_path = os.path.join(path, "*.csv")
df_list = []
for f in glob.glob(raw_files_path):
    print(f)
    df = pd.read_csv(f, index_col=None, header=0)
    df_list.append(df)
df = pd.concat(df_list, ignore_index = True)

C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2004_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2005_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2006_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2007_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2008_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2009_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2011_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2012_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2013_prices.csv
C:/Users/jacob.hester/Docume

In [3]:
# check data types
df.dtypes

Year              int64
Type             object
Weight_group     object
Hd Cnt           object
Jan             float64
Feb             float64
Mar             float64
Apr             float64
May             float64
Jun             float64
Jul             float64
Aug             float64
Sep             float64
Oct             float64
Nov             float64
Dec             float64
Avg             float64
dtype: object

In [4]:
# rename Hd Cnt col and change type 
# NOTE: column was read in as object type because of commas in head counts
df.rename(columns = {'Hd Cnt' : 'Hd_cnt'}, inplace = True)
df['Hd_cnt'] = df['Hd_cnt'].str.replace(',', '')
data_types_dictionary = {'Hd_cnt' : float}
df = df.astype(data_types_dictionary)

In [5]:
# check for missing values 
df.isnull().sum()

Year               0
Type               0
Weight_group       0
Hd_cnt             0
Jan             1546
Feb             1527
Mar             1468
Apr             1444
May             1533
Jun             1533
Jul             1685
Aug             1544
Sep             1557
Oct             1459
Nov             1553
Dec             1683
Avg                0
dtype: int64

In [6]:
# look at overall sales per year 
sales_per_yr_df = df.groupby(['Year'])['Hd_cnt'].sum()
sales_per_yr_df = pd.DataFrame(sales_per_yr_df)
sales_per_yr_df.reset_index(inplace = True)

# change Year to datetime type 
sales_per_yr_df['Year'] = pd.to_datetime(
    sales_per_yr_df['Year'].astype(str), format='%Y'
    )

In [7]:
# sales per year visual
# alt.renderers.enable('altair_viewer')

# add x index values for visual
x_year_domain = ['2003-01-01', '2019-01-01']
sales_per_year = alt.Chart(sales_per_yr_df).mark_bar(size = 20).encode(
    x = alt.X(
        'Year', 
        axis = alt.Axis(
            titleFontSize = 14
            ),
        scale = alt.Scale(domain = x_year_domain)
            ),
    y = alt.Y(
        'Hd_cnt', 
        axis = alt.Axis(
            title = 'Head Count',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Overall Sales Per Year",
                "subtitle" : "Beef Cattle Sales in Alabama"
                },
            width = 700
    ).configure_axisY(
        titleAngle = 0,
        titleX = -100,
    ).configure_axisX(
        titleX = 350,
        titleY = 30
    ).configure_title(
        fontSize = 18   
    )   
sales_per_year

alt.Chart(...)

In [8]:
# look at average overall prices per year
prices_per_yr_df = df.groupby(['Year'])['Avg'].mean()
prices_per_yr_df = pd.DataFrame(prices_per_yr_df)
prices_per_yr_df.reset_index(inplace = True)
prices_per_yr_df['Year'] = pd.to_datetime(
    prices_per_yr_df['Year'].astype(str), format='%Y'
    )
prices_per_yr_df.head()

prices_per_year = alt.Chart(prices_per_yr_df).mark_bar(size = 20).encode(
    x = alt.X(
        'Year', 
        axis = alt.Axis(
            titleFontSize = 14
            ),
        scale = alt.Scale(domain = x_year_domain)
            ),
    y = alt.Y(
        'Avg', 
        axis = alt.Axis(
            title = 'Average Price',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Average Price Per Year",
                "subtitle" : "Beef Cattle Sales in Alabama"
                },
            width = 700
    ).configure_axisY(
        titleAngle = 0,
        titleX = -100,
    ).configure_axisX(
        titleX = 350,
        titleY = 30
    ).configure_title(
        fontSize = 18   
    )
prices_per_year

alt.Chart(...)

In [9]:
# look at average prices per month 
price_per_month_df = pd.melt(
    df, 
    id_vars = ['Year'], 
    value_vars = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
)
# start by dropping missing values 
price_per_month_df = price_per_month_df.dropna()

price_per_month_df = price_per_month_df.groupby(['Year', 'variable'])['value'].mean()
price_per_month_df = pd.DataFrame(price_per_month_df)
price_per_month_df.reset_index(inplace = True)
price_per_month_df['Year'] = pd.to_datetime(
    price_per_month_df['Year'].astype(str), format='%Y'
)

In [10]:
# add column for month and year 
months_dict = {'Jan' : 1, 
               'Feb' : 2,
               'Mar' : 3, 
               'Apr' : 4, 
               'May' : 5, 
               'Jun' : 6, 
               'Jul' : 7, 
               'Aug' : 8, 
               'Sep' : 9, 
               'Oct' : 10, 
               'Nov' : 11, 
               'Dec' : 12
              }
price_per_month_df['month'] = price_per_month_df['variable'].map(months_dict)
price_per_month_df['year'] = price_per_month_df['Year'].dt.year

In [11]:
price_per_month_df.head()

,Year,variable,value,month,year
0,2004-01-01,Apr,98.259673,4,2004
1,2004-01-01,Aug,111.039098,8,2004
2,2004-01-01,Dec,105.845301,12,2004
3,2004-01-01,Feb,176.858529,2,2004
4,2004-01-01,Jan,125.400099,1,2004


In [12]:
price_per_month_df.dtypes

Year        datetime64[ns]
variable            object
value              float64
month                int64
year                 int64
dtype: object

In [13]:
price_per_month_df['day'] = 1
price_per_month_df.head()

,Year,variable,value,month,year,day
0,2004-01-01,Apr,98.259673,4,2004,1
1,2004-01-01,Aug,111.039098,8,2004,1
2,2004-01-01,Dec,105.845301,12,2004,1
3,2004-01-01,Feb,176.858529,2,2004,1
4,2004-01-01,Jan,125.400099,1,2004,1


In [14]:
price_per_month_df['date_col'] = pd.to_datetime(price_per_month_df[['month', 'year', 'day']])

In [15]:
print(price_per_month_df.head(25))
print('')
print(price_per_month_df.dtypes)

         Year variable       value  month  year  day   date_col
0  2004-01-01      Apr   98.259673      4  2004    1 2004-04-01
1  2004-01-01      Aug  111.039098      8  2004    1 2004-08-01
2  2004-01-01      Dec  105.845301     12  2004    1 2004-12-01
3  2004-01-01      Feb  176.858529      2  2004    1 2004-02-01
4  2004-01-01      Jan  125.400099      1  2004    1 2004-01-01
5  2004-01-01      Jul  110.345911      7  2004    1 2004-07-01
6  2004-01-01      Jun  106.885388      6  2004    1 2004-06-01
7  2004-01-01      Mar   96.656243      3  2004    1 2004-03-01
8  2004-01-01      May  103.132754      5  2004    1 2004-05-01
9  2004-01-01      Nov  105.206058     11  2004    1 2004-11-01
10 2004-01-01      Oct  105.829586     10  2004    1 2004-10-01
11 2004-01-01      Sep  107.097124      9  2004    1 2004-09-01
12 2005-01-01      Apr  116.129494      4  2005    1 2005-04-01
13 2005-01-01      Aug  107.362072      8  2005    1 2005-08-01
14 2005-01-01      Dec  111.994426     1

In [16]:
# prices per month over time visual
prices_per_month = alt.Chart(price_per_month_df).mark_point().encode(
    x = alt.X(
        'date_col', 
        axis = alt.Axis(
            title = 'Date',
            titleFontSize = 14
            ),
            ),
    y = alt.Y(
        'value', 
        axis = alt.Axis(
            title = 'Average Price',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Average Price Per Month",
                "subtitle" : "Average Monthly Beef Cattle Price in Alabama, 2004-2018"
                },
            width = 2000
    ).configure_axisY(
        titleAngle = 0,
        titleX = -100,
    ).configure_axisX(
        titleX = 1000,
        titleY = 30
    ).configure_title(
        fontSize = 18   
    )
prices_per_month

alt.Chart(...)

In [17]:
# look at what month is highest each year 
max_vals_dict = price_per_month_df.groupby(['year'])['value'].max().to_dict()
print(max_vals_dict)

{2004: 176.8585294117647, 2005: 116.51857723577231, 2006: 115.4780073800739, 2007: 105.3419305019305, 2008: 97.15622580645166, 2009: 89.99515789473686, 2011: 127.42837837837835, 2012: 155.7026724137931, 2013: 152.78152, 2014: 247.43409836065572, 2015: 246.97152173913048, 2016: 162.55872000000002, 2017: 136.1089705882353, 2018: 141.3530935251799}


In [18]:
price_per_month_df['max_price'] = price_per_month_df['year'].map(max_vals_dict)
print(price_per_month_df.head())

        Year variable       value  month  year  day   date_col   max_price
0 2004-01-01      Apr   98.259673      4  2004    1 2004-04-01  176.858529
1 2004-01-01      Aug  111.039098      8  2004    1 2004-08-01  176.858529
2 2004-01-01      Dec  105.845301     12  2004    1 2004-12-01  176.858529
3 2004-01-01      Feb  176.858529      2  2004    1 2004-02-01  176.858529
4 2004-01-01      Jan  125.400099      1  2004    1 2004-01-01  176.858529


In [19]:
price_per_month_df['max_price_month'] = np.where(price_per_month_df['value'] == price_per_month_df['max_price'], 1, 0)
print(price_per_month_df.head())

        Year variable       value  month  year  day   date_col   max_price  \
0 2004-01-01      Apr   98.259673      4  2004    1 2004-04-01  176.858529   
1 2004-01-01      Aug  111.039098      8  2004    1 2004-08-01  176.858529   
2 2004-01-01      Dec  105.845301     12  2004    1 2004-12-01  176.858529   
3 2004-01-01      Feb  176.858529      2  2004    1 2004-02-01  176.858529   
4 2004-01-01      Jan  125.400099      1  2004    1 2004-01-01  176.858529   

   max_price_month  
0                0  
1                0  
2                0  
3                1  
4                0  


In [20]:
prices_per_month = alt.Chart(price_per_month_df).mark_point().encode(
     color = 'max_price_month',
    x = alt.X(
        'date_col', 
        axis = alt.Axis(
            title = 'Date',
            titleFontSize = 14
            ),
            ),
    y = alt.Y(
        'value', 
        axis = alt.Axis(
            title = 'Average Price',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Average Price Per Month",
                "subtitle" : "Average Monthly Beef Cattle Price in Alabama, 2004-2018"
                },
            width = 2000
    )

text = (
    alt.Chart(price_per_month_df.query("max_price_month == 1"))
    .mark_text(dy=-15, color="red")
    .encode(x=alt.X("date_col"), y=alt.Y("value"), text=alt.Text("variable"))
)

(prices_per_month + text)

alt.LayerChart(...)